In [3]:
import warnings

warnings.filterwarnings("ignore")

In [10]:
import pyod
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyod.utils.data import evaluate_print
from sklearn.model_selection import train_test_split
from pyod.utils.utility import standardizer

In [11]:
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.iforest import IForest
from pyod.models.auto_encoder import AutoEncoder
import tensorflow
from pyod.models.deep_svdd import DeepSVDD

# run table(all methods/all data)

In [12]:
mnist=pd.read_csv("mnist.csv")
musk=pd.read_csv("musk.csv")
optdigit=pd.read_csv("optdigits.csv")
pendigits=pd.read_csv("pendigits.csv")
satimage_2=pd.read_csv("satimage-2.csv")
dataset={"mnist":mnist,"musk":musk,"optdigit":optdigit,"pendigits":pendigits,"satimage_2":satimage_2}


table_roc=pd.DataFrame()
table_recall=pd.DataFrame()

for name,data in dataset.items():
    print("第",name ,'個資料處理中....')
    print("")

    #降維
    import umap
    reducer = umap.UMAP(random_state=69,n_neighbors=20, min_dist=0.3, n_components=2, metric='manhattan')
    data_umap = reducer.fit_transform(data.iloc[:,0:-1])
    
    #split train/test
    X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,0:-1],data["outlier"], test_size=0.06, random_state=69)
    X_train_umap, X_test_umap, y_train, y_test = train_test_split(data_umap,data["outlier"], test_size=0.06, random_state=69)
    
    # standardizing data for processing
    X_train_norm, X_test_norm = standardizer(X_train, X_test)
    
    roc_df=pd.DataFrame()
    recall_df=pd.DataFrame()
    for time in range(5):
        
        #split train/valid
        X, X_valid_norm, y, y_valid = train_test_split(X_train_norm,y_train, test_size=0.25, random_state=time)
        X_umap, X_valid_norm_umap, y, y_valid = train_test_split(X_train_umap,y_train, test_size=0.25, random_state=time)

        # count outliers_ratio
        unique, counts = np.unique(y, return_counts=True)
        outliers_fraction=counts[1]/counts[0]

        #roc_list,recall_list
        roc_list=[]
        recall_list=[]

        classifiers = {'One-class SVM (OCSVM)':OCSVM(contamination=outliers_fraction),
                    'Local Outlier Factor (LOF)': LOF(contamination=outliers_fraction,n_neighbors=110),
                    'Isolation Forest': IForest(contamination=outliers_fraction,n_estimators=200,random_state=69),
                    'AutoEncoder': AutoEncoder(hidden_neurons=[2, 32,64,64, 32, 2],contamination=outliers_fraction,random_state=69,epochs=150),
                    'DeepSVDD':DeepSVDD(hidden_neurons=[X.shape[1],X.shape[1]*20], contamination=outliers_fraction,random_state=69,epochs=100,validation_size=0.05) }

        for clf_name, clf in classifiers.items():

            # Build the Model
            clf_name = clf_name
            clf = clf

            # Fit detector
            clf.fit(X[y == 0])
            # valid data
            y_pred = clf.predict(X) 
            y_scores = clf.decision_function(X)

            roc_list.append(evaluate_print(clf_name, y, y_scores)[0])
            recall_list.append(evaluate_print(clf_name, y, y_scores)[1])

        temp_df = pd.DataFrame(roc_list)
        temp_df.index = classifiers.keys()
        roc_df = pd.concat([roc_df, temp_df], axis=1)

        temp_df = pd.DataFrame(recall_list)
        temp_df.index = classifiers.keys()
        recall_df = pd.concat([recall_df, temp_df], axis=1)
        
    #average 5 times 
    table_roc=pd.concat([table_roc, roc_df.mean(axis=1)], axis=1)
    table_recall=pd.concat([table_recall,recall_df.mean(axis=1)], axis=1)

table_roc.columns=['mnist','musk','optdigits','pendigits','satimage-2']
table_recall.columns=['mnist','musk','optdigits','pendigits','satimage-2']

第 mnist 個資料處理中....

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
_________________________________________________________________
dropout_2 (Dropout)          (None, 2)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 

137/137 [==============================] - 0s 2ms/step - loss: 0.7157 - val_loss: 26.5706
Epoch 56/150
137/137 [==============================] - 0s 2ms/step - loss: 0.7148 - val_loss: 26.5702
Epoch 57/150
137/137 [==============================] - 0s 2ms/step - loss: 0.7137 - val_loss: 26.5697
Epoch 58/150
137/137 [==============================] - 0s 2ms/step - loss: 0.7129 - val_loss: 26.5692
Epoch 59/150
137/137 [==============================] - 0s 2ms/step - loss: 0.7121 - val_loss: 26.5686
Epoch 60/150
137/137 [==============================] - 0s 2ms/step - loss: 0.7114 - val_loss: 26.5680
Epoch 61/150
137/137 [==============================] - 0s 2ms/step - loss: 0.7106 - val_loss: 26.5674
Epoch 62/150
137/137 [==============================] - 0s 2ms/step - loss: 0.7100 - val_loss: 26.5669
Epoch 63/150
137/137 [==============================] - 0s 2ms/step - loss: 0.7094 - val_loss: 26.5663
Epoch 64/150
137/137 [==============================] - 0s 2ms/step - loss: 0.7088 - v

137/137 [==============================] - 0s 2ms/step - loss: 0.7014 - val_loss: 26.5590
Epoch 135/150
137/137 [==============================] - 0s 2ms/step - loss: 0.7014 - val_loss: 26.5590
Epoch 136/150
137/137 [==============================] - 0s 2ms/step - loss: 0.7014 - val_loss: 26.5589
Epoch 137/150
137/137 [==============================] - 0s 2ms/step - loss: 0.7014 - val_loss: 26.5589
Epoch 138/150
137/137 [==============================] - 0s 2ms/step - loss: 0.7014 - val_loss: 26.5589
Epoch 139/150
137/137 [==============================] - 0s 2ms/step - loss: 0.7014 - val_loss: 26.5589
Epoch 140/150
137/137 [==============================] - 0s 2ms/step - loss: 0.7014 - val_loss: 26.5589
Epoch 141/150
137/137 [==============================] - 0s 2ms/step - loss: 0.7014 - val_loss: 26.5589
Epoch 142/150
137/137 [==============================] - 0s 2ms/step - loss: 0.7014 - val_loss: 26.5589
Epoch 143/150
137/137 [==============================] - 0s 3ms/step - loss: 0

145/145 [==============================] - 0s 3ms/step - loss: 2.2287 - val_loss: 41.1255
Epoch 49/100
145/145 [==============================] - 0s 3ms/step - loss: 2.2287 - val_loss: 41.1330
Epoch 50/100
145/145 [==============================] - 0s 3ms/step - loss: 2.2287 - val_loss: 41.1375
Epoch 51/100
145/145 [==============================] - 0s 3ms/step - loss: 2.2287 - val_loss: 41.1414
Epoch 52/100
145/145 [==============================] - 0s 3ms/step - loss: 2.2287 - val_loss: 41.1445
Epoch 53/100
145/145 [==============================] - 0s 3ms/step - loss: 2.2287 - val_loss: 41.1472
Epoch 54/100
145/145 [==============================] - 0s 3ms/step - loss: 2.2287 - val_loss: 41.1492
Epoch 55/100
145/145 [==============================] - 0s 3ms/step - loss: 2.2287 - val_loss: 41.1510
Epoch 56/100
145/145 [==============================] - 0s 3ms/step - loss: 2.2287 - val_loss: 41.1529
Epoch 57/100
145/145 [==============================] - 0s 3ms/step - loss: 2.2287 - v

137/137 [==============================] - 0s 3ms/step - loss: 2.9941 - val_loss: 6.0236
Epoch 3/150
137/137 [==============================] - 0s 3ms/step - loss: 2.2423 - val_loss: 7.0690
Epoch 4/150
137/137 [==============================] - 0s 3ms/step - loss: 1.8700 - val_loss: 8.0549
Epoch 5/150
137/137 [==============================] - 0s 3ms/step - loss: 1.6452 - val_loss: 9.0433
Epoch 6/150
137/137 [==============================] - 0s 2ms/step - loss: 1.4812 - val_loss: 9.8905
Epoch 7/150
137/137 [==============================] - 0s 3ms/step - loss: 1.3553 - val_loss: 10.7334
Epoch 8/150
137/137 [==============================] - 0s 3ms/step - loss: 1.2610 - val_loss: 11.5181
Epoch 9/150
137/137 [==============================] - 0s 3ms/step - loss: 1.1921 - val_loss: 12.2203
Epoch 10/150
137/137 [==============================] - 0s 3ms/step - loss: 1.1245 - val_loss: 13.0827
Epoch 11/150
137/137 [==============================] - 0s 2ms/step - loss: 1.0779 - val_loss: 13.

...

120/120 [==============================] - 1s 2ms/step - loss: 5.5754 - val_loss: 2.4904
Epoch 2/100
120/120 [==============================] - 0s 1ms/step - loss: 1.9711 - val_loss: 1.6125
Epoch 3/100
120/120 [==============================] - 0s 2ms/step - loss: 1.4486 - val_loss: 1.3360
Epoch 4/100
120/120 [==============================] - 0s 2ms/step - loss: 1.2605 - val_loss: 1.2121
Epoch 5/100
120/120 [==============================] - 0s 1ms/step - loss: 1.1740 - val_loss: 1.1489
Epoch 6/100
120/120 [==============================] - 0s 1ms/step - loss: 1.1287 - val_loss: 1.1145
Epoch 7/100
120/120 [==============================] - 0s 2ms/step - loss: 1.1034 - val_loss: 1.0945
Epoch 8/100
120/120 [==============================] - 0s 2ms/step - loss: 1.0883 - val_loss: 1.0823
Epoch 9/100
120/120 [==============================] - 0s 2ms/step - loss: 1.0790 - val_loss: 1.0747
Epoch 10/100
120/120 [==============================] - 0s 2ms/step - loss: 1.0729 - val_loss: 1.0699
E

120/120 [==============================] - 0s 1ms/step - loss: 1.0550 - val_loss: 1.0551
Epoch 82/100
120/120 [==============================] - 0s 1ms/step - loss: 1.0550 - val_loss: 1.0551
Epoch 83/100
120/120 [==============================] - 0s 2ms/step - loss: 1.0550 - val_loss: 1.0551
Epoch 84/100
120/120 [==============================] - 0s 1ms/step - loss: 1.0550 - val_loss: 1.0551
Epoch 85/100
120/120 [==============================] - 0s 2ms/step - loss: 1.0550 - val_loss: 1.0551
Epoch 86/100
120/120 [==============================] - 0s 1ms/step - loss: 1.0550 - val_loss: 1.0551
Epoch 87/100
120/120 [==============================] - 0s 1ms/step - loss: 1.0550 - val_loss: 1.0551
Epoch 88/100
120/120 [==============================] - 0s 1ms/step - loss: 1.0550 - val_loss: 1.0551
Epoch 89/100
120/120 [==============================] - 0s 1ms/step - loss: 1.0550 - val_loss: 1.0551
Epoch 90/100
120/120 [==============================] - 0s 1ms/step - loss: 1.0550 - val_loss: 

Epoch 36/150
114/114 [==============================] - 0s 2ms/step - loss: 1.0307 - val_loss: 1.1082
Epoch 37/150
114/114 [==============================] - 0s 2ms/step - loss: 1.0271 - val_loss: 1.1059
Epoch 38/150
114/114 [==============================] - 0s 2ms/step - loss: 1.0260 - val_loss: 1.1039
Epoch 39/150
114/114 [==============================] - 0s 2ms/step - loss: 1.0236 - val_loss: 1.1020
Epoch 40/150
114/114 [==============================] - 0s 2ms/step - loss: 1.0221 - val_loss: 1.1003
Epoch 41/150
114/114 [==============================] - 0s 2ms/step - loss: 1.0194 - val_loss: 1.0985
Epoch 42/150
114/114 [==============================] - 0s 2ms/step - loss: 1.0193 - val_loss: 1.0971
Epoch 43/150
114/114 [==============================] - 0s 2ms/step - loss: 1.0181 - val_loss: 1.0957
Epoch 44/150
114/114 [==============================] - 0s 2ms/step - loss: 1.0156 - val_loss: 1.0941
Epoch 45/150
114/114 [==============================] - 0s 2ms/step - loss: 1.0147

Epoch 116/150
114/114 [==============================] - 0s 2ms/step - loss: 0.9935 - val_loss: 1.0642
Epoch 117/150
114/114 [==============================] - 0s 2ms/step - loss: 0.9935 - val_loss: 1.0641
Epoch 118/150
114/114 [==============================] - 0s 2ms/step - loss: 0.9935 - val_loss: 1.0641
Epoch 119/150
114/114 [==============================] - 0s 2ms/step - loss: 0.9935 - val_loss: 1.0640
Epoch 120/150
114/114 [==============================] - 0s 2ms/step - loss: 0.9935 - val_loss: 1.0640
Epoch 121/150
114/114 [==============================] - 0s 2ms/step - loss: 0.9935 - val_loss: 1.0639
Epoch 122/150
114/114 [==============================] - 0s 2ms/step - loss: 0.9934 - val_loss: 1.0639
Epoch 123/150
114/114 [==============================] - 0s 2ms/step - loss: 0.9934 - val_loss: 1.0638
Epoch 124/150
114/114 [==============================] - 0s 2ms/step - loss: 0.9934 - val_loss: 1.0638
Epoch 125/150
114/114 [==============================] - 0s 2ms/step - lo

120/120 [==============================] - 0s 1ms/step - loss: 1.0539 - val_loss: 1.0541
Epoch 31/100
120/120 [==============================] - 0s 1ms/step - loss: 1.0539 - val_loss: 1.0540
Epoch 32/100
120/120 [==============================] - 0s 1ms/step - loss: 1.0538 - val_loss: 1.0539
Epoch 33/100
120/120 [==============================] - 0s 1ms/step - loss: 1.0537 - val_loss: 1.0539
Epoch 34/100
120/120 [==============================] - 0s 1ms/step - loss: 1.0537 - val_loss: 1.0539
Epoch 35/100
120/120 [==============================] - 0s 1ms/step - loss: 1.0537 - val_loss: 1.0538
Epoch 36/100
120/120 [==============================] - 0s 1ms/step - loss: 1.0536 - val_loss: 1.0538
Epoch 37/100
120/120 [==============================] - 0s 1ms/step - loss: 1.0536 - val_loss: 1.0538
Epoch 38/100
120/120 [==============================] - 0s 1ms/step - loss: 1.0536 - val_loss: 1.0537
Epoch 39/100
120/120 [==============================] - 0s 1ms/step - loss: 1.0536 - val_loss: 

In [13]:
table_roc

,mnist,musk,optdigits,pendigits,satimage-2
One-class SVM (OCSVM),0.87772,1.00000,0.60342,0.96430,0.99636
Local Outlier Factor (LOF),0.93652,1.00000,0.91218,0.97948,0.99702
Isolation Forest,0.86552,0.98884,0.84948,0.97602,0.99244
AutoEncoder,0.90326,1.00000,0.56820,0.94272,0.97804
DeepSVDD,0.71202,1.00000,0.52062,0.40054,0.51184


In [14]:
table_recall

,mnist,musk,optdigits,pendigits,satimage-2
One-class SVM (OCSVM),0.44394,1.00000,0.00000,0.44198,0.92870
Local Outlier Factor (LOF),0.61770,1.00000,0.01352,0.44326,0.85148
Isolation Forest,0.41854,0.68452,0.06400,0.49382,0.86588
AutoEncoder,0.55492,1.00000,0.00000,0.34414,0.83726
DeepSVDD,0.52176,1.00000,0.28556,0.01622,0.09106


In [15]:
table_roc.to_csv("table_roc_70.csv")
table_recall.to_csv("table_recall_70.csv")

# part data

In [17]:
data1=pd.read_csv("mnist.csv")
data2=pd.read_csv("musk.csv")
data3=pd.read_csv("optdigits.csv")
data4=pd.read_csv("pendigits.csv")
data5=pd.read_csv("satimage-2.csv")

In [18]:
print("data1：",data1["outlier"].count())
print("data2：",data2["outlier"].count())
print("data3：",data3["outlier"].count())
print("data4：",data4["outlier"].count())
print("data5：",data5["outlier"].count())

data1： 7603
data2： 3062
data3： 5216
data4： 6870
data5： 5803


In [19]:
print("data1：",round(data1["outlier"].mean(),3))
print("data2：",round(data2["outlier"].mean(),3))
print("data3：",round(data3["outlier"].mean(),3))
print("data4：",round(data4["outlier"].mean(),3))
print("data5：",round(data5["outlier"].mean(),3))

data1： 0.092
data2： 0.032
data3： 0.029
data4： 0.023
data5： 0.012


In [ ]:
import umap
reducer = umap.UMAP(random_state=69,n_neighbors=20, min_dist=0.3, n_components=2, metric='manhattan')
data_umap = reducer.fit_transform(data.iloc[:,0:-1])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,0:-1],data["outlier"], test_size=0.06, random_state=69)
X_train_umap, X_test_umap, y_train, y_test = train_test_split(data_umap,data["outlier"], test_size=0.06, random_state=69)

In [ ]:
#標準化影響one class
from pyod.utils.utility import standardizer
# standardizing data for processing
X_train_norm, X_test_norm = standardizer(X_train, X_test)

In [20]:
mnist=pd.read_csv("mnist.csv")
musk=pd.read_csv("musk.csv")
optdigit=pd.read_csv("optdigits.csv")
pendigits=pd.read_csv("pendigits.csv")
satimage_2=pd.read_csv("satimage-2.csv")
dataset={"mnist":mnist,"musk":musk,"optdigit":optdigit,"pendigits":pendigits,"satimage_2":satimage_2}


table_roc=pd.DataFrame()
table_recall=pd.DataFrame()

for name,data in dataset.items():
    print("第",name ,'個資料處理中....')
    print("")

    #降維
    import umap
    reducer = umap.UMAP(random_state=69,n_neighbors=20, min_dist=0.3, n_components=2, metric='manhattan')
    data_umap = reducer.fit_transform(data.iloc[:,0:-1])
    
    #split train/test
    X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,0:-1],data["outlier"], test_size=0.06, random_state=69)
    X_train_umap, X_test_umap, y_train, y_test = train_test_split(data_umap,data["outlier"], test_size=0.06, random_state=69)
    
    # standardizing data for processing
    X_train_norm, X_test_norm = standardizer(X_train, X_test)
        
    #split train/valid
    X, X_valid_norm, y, y_valid = train_test_split(X_train_norm,y_train, test_size=0.25, random_state=time)
    X_umap, X_valid_norm_umap, y, y_valid = train_test_split(X_train_umap,y_train, test_size=0.25, random_state=time)

    # count outliers_ratio
    unique, counts = np.unique(y, return_counts=True)
    outliers_fraction=counts[1]/counts[0]

    classifiers = {'One-class SVM (OCSVM)':OCSVM(contamination=outliers_fraction),
                'Local Outlier Factor (LOF)': LOF(contamination=outliers_fraction,n_neighbors=110),
                'Isolation Forest': IForest(contamination=outliers_fraction,n_estimators=200,random_state=69),
                'AutoEncoder': AutoEncoder(hidden_neurons=[2, 32,64,64, 32, 2],contamination=outliers_fraction,random_state=69,epochs=150),
                'DeepSVDD':DeepSVDD(hidden_neurons=[data.shape[1],data.shape[1]*20], contamination=outliers_fraction,random_state=69,epochs=100,validation_size=0.05) }

    for clf_name, clf in classifiers.items():

        # Build the Model
        clf_name = clf_name
        clf = clf

        # Fit detector
        clf.fit(X[y == 0])
        # valid data
        y_pred = clf.predict(X) 
        y_scores = clf.decision_function(X)

        roc_list.append(evaluate_print(clf_name, y, y_scores)[0])
        recall_list.append(evaluate_print(clf_name, y, y_scores)[1])

        temp_df = pd.DataFrame(roc_list)
        temp_df.index = classifiers.keys()
        roc_df = pd.concat([roc_df, temp_df], axis=1)

        temp_df = pd.DataFrame(recall_list)
        temp_df.index = classifiers.keys()
        recall_df = pd.concat([recall_df, temp_df], axis=1)
        
    #average 5 times 
    table_roc=pd.concat([table_roc, roc_df.mean(axis=1)], axis=1)
    table_recall=pd.concat([table_recall,recall_df.mean(axis=1)], axis=1)

table_roc.columns=['mnist','musk','optdigits','pendigits','satimage-2']
table_recall.columns=['mnist','musk','optdigits','pendigits','satimage-2']

第 mnist 個資料處理中....



KeyboardInterrupt: 